In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc='My bar!')
from collections import defaultdict

/cm/shared/apps/spack/gpu/opt/spack/linux-centos8-skylake_avx512/gcc-8.3.1/anaconda3-2020.11-bsn4npoxyw7jzz7fajncek3bvdoaa5wv/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
genes = pd.read_csv("Homo_sapiens.GRCh38.108.gtf", sep = "\t", comment="#", header=None)
genes[0] = "chr" + genes[0].astype(str)


def find_genes(row):
    gene_df = genes[(genes[0] == row['CHROM']) & (genes[3] <= row['POS']) & 
                    (genes[4] >= row['POS'])]
    if len(gene_df) == 0:
        return None
    gene_df_info = gene_df[8].str.split(";")
    gene_set = set()
    for info in list(gene_df_info):
        for field in info:
            if "gene_name" in field:
                name = field.strip().replace("gene_name ","").replace('"','')
                gene_set.add(name)
    if len(gene_set) == 0:
        return None
                
    return ",".join(list(gene_set))

/cm/shared/apps/spack/gpu/opt/spack/linux-centos8-skylake_avx512/gcc-8.3.1/anaconda3-2020.11-bsn4npoxyw7jzz7fajncek3bvdoaa5wv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
all_expansions = pd.DataFrame(columns = ['CHROM', 'POS', 'PERIOD', 'MOTIF', 'outlier_threshold', 'AFR_freq',
       'NON_AFR_freq'])
for i in range(1,23):
    expansions = pd.read_csv(f"{i}_expansions.csv")
    all_expansions = pd.concat([expansions, all_expansions])
    
    
all_expansions['gene'] = all_expansions.apply(lambda row: find_genes(row), axis = 1)

In [8]:
pedigree = pd.read_csv("/expanse/projects/gymreklab/helia/TR_1000G/1000G.ped", delim_whitespace=True)
pedigree = pedigree[['SampleID','Superpopulation', 'Population']]

samp_to_subpop = pd.Series(pedigree[pedigree['Superpopulation'] == "AFR"].Population.values,index=pedigree[pedigree['Superpopulation'] == "AFR"].SampleID).to_dict()
samp_to_pop = pd.Series(pedigree.Superpopulation.values,index=pedigree.SampleID).to_dict()
H3Africa_names = pd.read_csv("/expanse/projects/gymreklab/helia/H3Africa/names/H3A_Baylor_sample_country.txt", header=None, delim_whitespace=True)

for index,row in H3Africa_names.iterrows():
    samp_to_pop[row[0]] = "H3Africa"
    samp_to_subpop[row[0]] = row[1]
    
african_pop = set(list(samp_to_subpop.values()))

In [8]:
# significant_expansion = all_expansions[(all_expansions['outlier_threshold'] > 1) & 
#                (all_expansions['PERIOD'] > 2)].sort_values('outlier_threshold', ascending = False)

# significant_expansion.to_csv("significant_expansions.csv",  index = False)

In [10]:
def find_freqs(row):
    chrom = row['CHROM'].replace("chr", "")
    header = []
    with open(f"/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/diff/{chrom}_diff.txt") as f:
        for line in f:
            if line.startswith("chr"):
                    f.close()
                    break
            header.append(line.strip())

    addr=f"/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/diff/{chrom}_diff.txt"

    pos = row['POS']
    x=!grep -m 1 -w $pos $addr
    gbs = x[0].split("\t")[4:]
    dict_ = dict(zip(header, gbs))
    
    for x in dict_:
        dict_[x] = dict_[x].split("/")
        modified_gbs = []
        for s in dict_[x]:
            if s != "." and s != "":
                modified_gbs.append(int(s)/ row['PERIOD'])
                
        dict_[x] = modified_gbs
        
        
        
    pop_freqs = {}
    for pop in ['AFR', 'H3Africa']:
        gbs = []
        for samp in dict_:
            if samp in samp_to_pop:
                if samp_to_pop[samp] == pop:
                    gbs.extend(dict_[samp])
                
        if len(gbs) == 0:
            pop_freqs[pop] = -1
        else:
            above_threshold = round(len([x for x in gbs if x not in [".",""] \
                       and x > row['outlier_threshold']]) / len([x for x in gbs if x not in [".",""]]),3)
            pop_freqs[pop] = above_threshold
                
                
            
    
    
    
    return pop_freqs

all_expansions[['AFR_freqs', 
                        'H3Africa_freqs']] = all_expansions.apply(lambda row: find_freqs(row), 
                                                                         axis = 1, result_type="expand")
all_expansions

,CHROM,POS,PERIOD,MOTIF,outlier_threshold,AFR_freq,NON_AFR_freq,gene,AFR_freqs,H3Africa_freqs
0,chr22,17976536,2,AC,6.0,0.027958,0.000000,MICAL3,0.028,0.029
1,chr22,20888311,2,AC,8.0,0.077357,0.000866,SNAP29,0.071,0.093
2,chr22,22722990,2,AG,9.0,0.042406,0.000000,None,0.049,0.039
3,chr22,24042817,2,AC,16.0,0.032510,0.000650,CABIN1,0.036,0.024
4,chr22,24496373,2,AC,7.0,0.018623,0.000435,UPB1,0.017,0.023
...,...,...,...,...,...,...,...,...,...,...
173,chr1,247419145,2,AT,4.5,0.027386,0.000655,NLRP3,0.030,0.021
174,chr1,247919144,2,AC,5.0,0.081789,0.001732,None,0.077,0.093
175,chr1,248249150,2,AT,8.0,0.055197,0.001083,None,0.058,0.047
176,chr1,248684754,2,AT,6.0,0.072522,0.002599,None,0.074,0.069


In [30]:
all_expansions_both = all_expansions[(all_expansions['AFR_freq'] < 0.01) | 
                                     (all_expansions['AFR_freqs'] > 0.01) & 
                                     (all_expansions['H3Africa_freqs'] > 0.01)]
all_expansions_both['outlier_threshold'] = all_expansions_both['outlier_threshold'].round(1)
all_expansions_both['NON_AFR_freq'] = all_expansions_both['NON_AFR_freq'].round(3)
all_expansions_both['AFR_freq'] = all_expansions_both['AFR_freq'].round(3)
all_expansions_both.columns = ['CHROM','POS','PERIOD','MOTIF','OUTLIER_THRESH',
                               'ALL_AFR_FREQ','NON_AFR_FREQ','GENE','1KG_AFR_FREQ','H3Africa_FREQ']
all_expansions_both
all_expansions_both.sort_values('OUTLIER_THRESH', ascending=False).to_csv('Large_Expansions.csv', index = False)

<ipython-input-30-c7562cd2a9ae>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_expansions_both['outlier_threshold'] = all_expansions_both['outlier_threshold'].round(1)
<ipython-input-30-c7562cd2a9ae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_expansions_both['NON_AFR_freq'] = all_expansions_both['NON_AFR_freq'].round(3)
<ipython-input-30-c7562cd2a9ae>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [33]:
all_expansions_both.sort_values('OUTLIER_THRESH', ascending=False)

,CHROM,POS,PERIOD,MOTIF,OUTLIER_THRESH,ALL_AFR_FREQ,NON_AFR_FREQ,GENE,1KG_AFR_FREQ,H3Africa_FREQ
32,chr17,51831667,3,AGC,65.0,0.016,0.001,CA10,0.014,0.022
58,chr1,77887912,3,AAG,39.7,0.140,0.012,NEXN-AS1,0.141,0.149
23,chr13,49441221,2,AT,32.0,0.001,0.013,CAB39L,0.001,0.000
4,chr18,2956311,2,AC,32.0,0.015,0.000,LPIN2,0.013,0.019
16,chr1,20599617,5,AAAAT,24.0,0.039,0.001,CDA,0.035,0.047
...,...,...,...,...,...,...,...,...,...,...
78,chr10,97239551,2,AC,4.5,0.013,0.000,"ARHGAP19-SLIT1,ARHGAP19",0.013,0.014
10,chr7,15454357,2,AC,4.5,0.066,0.004,AGMO,0.071,0.053
147,chr6,169607394,2,AC,4.5,0.044,0.000,WDR27,0.041,0.052
40,chr11,55450915,2,AT,4.5,0.185,0.002,None,0.171,0.221


In [35]:
all_expansions_both[all_expansions_both['ALL_AFR_FREQ'] < all_expansions_both['NON_AFR_FREQ']].sort_values("OUTLIER_THRESH")

,CHROM,POS,PERIOD,MOTIF,OUTLIER_THRESH,ALL_AFR_FREQ,NON_AFR_FREQ,GENE,1KG_AFR_FREQ,H3Africa_FREQ
53,chr17,66290887,2,AC,5.0,0.006,0.068,None,0.008,0.003
69,chr18,73883512,2,AG,5.0,0.000,0.018,None,0.000,0.000
70,chr18,73883530,2,AG,5.0,0.000,0.017,None,0.000,0.001
71,chr6,77981741,2,AG,5.0,0.001,0.082,None,0.001,0.000
29,chr9,25540102,2,AG,5.0,0.014,0.154,None,0.014,0.014
...,...,...,...,...,...,...,...,...,...,...
96,chr8,139159483,2,AT,20.0,0.000,0.013,None,0.000,0.000
62,chr9,92567372,2,AT,21.0,0.000,0.027,CENPP,0.000,0.000
101,chr12,113879432,2,AT,21.0,0.000,0.037,RBM19,0.000,0.000
102,chr1,187695913,2,AT,22.0,0.000,0.020,None,0.000,0.000
